<a href="https://colab.research.google.com/github/min01118/interview/blob/main/%EC%9D%B4%EC%8A%A4%ED%8A%B8%EC%86%8C%ED%94%84%ED%8A%B8%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests

In [ ]:
import requests
import time
import re
import urllib.parse

In [ ]:
#필요한 데이터
api_key=""
id = ""
password: str =""

In [ ]:
#데이터셋 리스트

base_url = "https://api.aihub.or.kr"
list_url = f'{base_url}/info/dataset.do'
response=requests.get(list_url)
datasets = [i.split(",")[0] for i in response.text.split("\n") if "뉴스 대본 및 앵커 음성 데이터" in i]
datasets

['71557']

In [ ]:
#데이터 트리정보 가져오기
dataset_data_url = f"{base_url}/info/{datasets[0]}.do"
response_k = requests.get(dataset_data_url)
tree_text = response_k.text
tree_text.split("\n")

['The contents are encoded in UTF-8 including Korean characters. ',
 'If the following contents are not output normally, ',
 'Please modify the character information of the OS. ',
 '=================공지사항=================== ',
 '========================================== ',
 '',
 '    └─138.뉴스 대본 및 앵커 음성 데이터',
 '        └─01-1.정식개방데이터',
 '            ├─Training',
 '            │  ├─01.원천데이터',
 '            │  │  ├─TS.z01 | 100 GB | 470564',
 '            │  │  ├─TS.z02 | 100 GB | 470565',
 '            │  │  └─TS.zip | 2 GB | 470566',
 '            │  └─02.라벨링데이터',
 '            │      └─TL.zip | 244 MB | 470567',
 '            └─Validation',
 '                ├─01.원천데이터',
 '                │  └─VS.zip | 25 GB | 470568',
 '                └─02.라벨링데이터',
 '                    └─VL.zip | 28 MB | 470569',
 '',
 '']

In [ ]:
#Tree Maker
import re

class Component :
  def __init__(self ,name : str,depth:int) -> None:
    self.name = name
    self.depth = depth
  def visit(self, do ):
    do(self)

class Leaf(Component) :
  def __init__(self, name: str, depth:int, parent : Component|None) -> None:
    super().__init__(name,depth)
    self.parent = parent

class ListTypeTree(Leaf) :
  def __init__(self, name: str, depth:int, parent : Component|None) -> None:
    super().__init__(name,depth,parent)
    self.children : list[Leaf] = []

  def add_child(self, child : Leaf):
    self.children.append(child)

  def visit(self, do ):
    do(self)
    for i in self.children:
      i.visit(do)



class AiHubTreeMaker :
  def __init__(self,text) -> None:
    self.root_parent = ListTypeTree("root",0,None)
    self.current_parent = self.root_parent
    self.text =text
    for i in  text.split("\n") :
          self.__find_depth_and_add(i)

  def __add_child(self ,line : str):
     self.current_parent.add_child(ListTypeTree(line,self.current_parent.depth+1,self.current_parent))

  def __check_depth(self, line : str):
    return len(re.split(r"(├─|└─)",line)[0].replace("│","  "))/4

  def __change_parent(self, depth:int,line:str):
    if  depth - 1 < self.current_parent.depth :
      self.current_parent = self.current_parent.parent
      self.__change_parent(depth,line)
    elif depth - 1 > self.current_parent.depth :
      self.current_parent = self.current_parent.children[-1]

  def __find_depth_and_add(self, line:str):
    if not re.match(r"^.*(├─|└─)",line) :
      return
    depth =  self.__check_depth(line)
    self.__change_parent(depth,line)
    self.__add_child(line)


  def visitAll(self,do):
    self.root_parent.visit(do)

In [ ]:
# 라벨링 데이터 만 가져오기
returnFileList:list[tuple] = []
def check(data : ListTypeTree) :
  if "라벨링데이터" in data.name:
      for i in data.children:
        returnFileList.append((re.findall(r'[a-zA-Z]+',data.parent.name.strip())[0].lower(),i.name.split("|")[-1]))
AiHubTreeMaker(tree_text).visitAll(check)
returnFileList

[('training', ' 470567'), ('validation', ' 470569')]

In [ ]:
#파일 다운로드 시도 1 --------------------------실패 -------------------------
#아마존에 프록시 서버 구성 SQUID
#한국 서버가 필요합니다. 아마존 서울 서버는 해외서버로 인식됨
proxies = "http://ec2-15-164-214-40.ap-northeast-2.compute.amazonaws.com:3128"
#curl download
download_url : str= f"{base_url}/down/0.5"
def download_file(file_name:str,file_key,proxies=None):
  api_key_str = f"\"apikey:{api_key}\""
  url = f"\"{download_url}/{datasets[0].strip()}.do?fileSn={file_key.strip()}\""
  file_name = f"\"{file_name}.tar\""
  if proxies is None:
    !curl  -L -C - -o {file_name} -H {api_key_str}  {url}
  else :
    !curl  -L -C - -o {file_name} -x {proxies}  -H {api_key_str}  {url}

#no proxy
for i in returnFileList:
  download_file(i[0],i[1])

#with proxy
for i in returnFileList:
  download_file(i[0],i[1],proxies)

!cat training.tar
!cat validation.tar

** Resuming transfer from byte position 82
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0    82    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
** Resuming transfer from byte position 82
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0    82    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
** Resuming transfer from byte position 82
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0    82    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
** Resuming transfer from byte position 82
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dloa

In [ ]:
import requests
# 파일 다운로드 시도 1-1----------------------실패--------------------
# request 를 이용
#아마존에 프록시 서버 구성 SQUID
#한국 서버가 필요합니다. 아마존 서울 서버는 해외서버로 인식됨

proxies = {
    'http' : "http://ec2-15-164-214-40.ap-northeast-2.compute.amazonaws.com:3128",
    'https': "http://ec2-15-164-214-40.ap-northeast-2.compute.amazonaws.com:3128"
}
download_url : str= f"{base_url}/down/0.5"

def download_file_with_request(file_name:str,file_key:str ,proxies=None):
    headers = {'apikey': api_key}
    url = f"{download_url}/{datasets[0].strip()}.do?fileSn={file_key.strip()}"
    with requests.get(url, headers=headers, stream=True, proxies=proxies) as response:
        try:
          response.raise_for_status()
          file_name = f"\"{file_name}.tar\""
          with open(file_name, 'wb') as file:
              for chunk in response.iter_content(chunk_size=8192):
                  if chunk:
                      file.write(chunk)
        except :
            print(response.status_code)
            print(response.text)


for i in returnFileList:
  download_file_with_request(i[0],i[1],proxies)


502
AI 허브는 해외에서의 데이터 다운로드를 제한하고 있습니다.

502
AI 허브는 해외에서의 데이터 다운로드를 제한하고 있습니다.



In [ ]:
# 파일 다운로드 시도 2--------------------실패----------------------------------------------------
# 로그인 및 웹페이지 시도



login_url = "https://aihub.or.kr/loginProc.do"
login_data =f'referer=&emplyrId={urllib.parse.quote(id)}&password={urllib.parse.quote(password)}'
javascript_url = "https://aihub.or.kr/apishell/dwld.do"
javascript_data = {
    'dataSetSn': '71557',
    'fileSn': '470567'
}
headers = {
    'Accept': 'application/json',
    "content-type":"application/x-www-form-urlencoded; charset=UTF-8"
}

session = requests.Session()
response1 = session.post(login_url,headers=headers, data=login_data)
print(response1.text)
response2 = session.post(javascript_url, data=javascript_data)
response2.text

{"success":true,"returnURL":"/index.do","rstVal":"N"}


"<script>alert('AI 허브는 해외에서의 데이터 다운로드를 제한하고 있습니다.');window.close();</script>\n"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#tar 풀기
!mkdir out
!rm -rf out/*
!mkdir out/validation
!mkdir out/training
!tar -xvf  /content/drive/MyDrive/validation.tar -C ./out/validation
!tar -xvf  /content/drive/MyDrive/training.tar -C ./out/training

mkdir: cannot create directory ‘out’: File exists
138.뉴스_대본_및_앵커_음성_데이터/01-1.정식개방데이터/Validation/02.라벨링데이터/VL.zip.part0
138.뉴스_대본_및_앵커_음성_데이터/01-1.정식개방데이터/Training/02.라벨링데이터/TL.zip.part0


In [ ]:
#병합
import os
!cp out/validation/138.뉴스_대본_및_앵커_음성_데이터/01-1.정식개방데이터/Validation/02.라벨링데이터/VL.zip.part0  out/validation/VL.zip
!cp out/training/138.뉴스_대본_및_앵커_음성_데이터/01-1.정식개방데이터/Training/02.라벨링데이터/TL.zip.part0 out/training/TL.zip



In [ ]:
#압축 풀기
!mkdir out/validation/out
!rm -rf out/validation/out/*
!unzip -qq out/validation/VL.zip -d out/validation/out
!mkdir out/training/out
!rm -rf out/training/out/*
!unzip -qq out/training/TL.zip -d out/training/out

In [ ]:
#making text_only
import os
import json

def find_all_json_files(directory):
    json_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.json'):
                json_files.append(os.path.join(root, file))
    return json_files

def make_text_only(directory):
  json_files = find_all_json_files(f"{directory}/out")
  !rm -f {directory}/text_only.txt
  !touch {directory}/text_only.txt
  with open(f"{directory}/text_only.txt", 'a') as output_file:
    for json_file in json_files:
        with open(json_file, 'r') as input_file:
            text = input_file.read()
            data = json.loads(text)
            output_file.write(f'{data["script"]["id"]}:{data["script"]["text"]}\n')

make_text_only("out/validation")
make_text_only("out/training")


In [ ]:
import re

def make_text_preprocessed(directory):
  ! rm -f {directory}/text_preprocessed.txt
  ! touch {directory}/text_preprocessed.txt
  with open(f"{directory}/text_preprocessed.txt",'a') as output_file:
    with open(f"{directory}/text_only.txt", 'r') as input_file:
      lines = input_file.readlines()
      for i in lines:
        name=i.split(":")[0]
        i = re.sub(r'\n$',"",i).replace(f'{name}:',"")
        find_text = re.findall(r'\(.*?\)/\(.*?\)',i)
        first = i
        second = i
        for k in find_text :
          k_list=re.findall(r'\((.*?)\)/\((.*?)\)',k)[0]
          first = first.replace(k,k_list[0])
          second = second.replace(k,k_list[1])
        output_file.write(f'{name}:{first} ${second}\n')
make_text_preprocessed("out/validation")
make_text_preprocessed("out/training")

In [ ]:
!tail -n 20 out/validation/text_preprocessed.txt

YTNPO484:앞서 지난달 20 일 YTN 이 국회와 정당이 수어 통역을 제공하지 않아 의원 개인이 수어 통역사를 고용해야 하는 현실을 보도한 직후 박병석 국회의장은 국회사무처에 개선 방안을 찾아보라고 지시한 것으로 전해졌습니다. $앞서 지난달 이십 일 와이 티 엔 이 국회와 정당이 수어 통역을 제공하지 않아 의원 개인이 수어 통역사를 고용해야 하는 현실을 보도한 직후 박병석 국회의장은 국회사무처에 개선 방안을 찾아보라고 지시한 것으로 전해졌습니다.
YTNPO484:국회사무처는 `국민 누구와도 소통하는 국회` 를 만들기 위해 소통관 기자회견장 수어 통역 지원 계획을 세우고 다음 달부터 시범 사업에 들어갈 예정입니다. $국회사무처는 `국민 누구와도 소통하는 국회` 를 만들기 위해 소통관 기자회견장 수어 통역 지원 계획을 세우고 다음 달부터 시범 사업에 들어갈 예정입니다.
YTNPO484:지난달 YTN 의 `수어 소통 없는 국회 소통관` 보도와 관련해 앞으로는 국회 기자회견에도 코로나 19 정부 브리핑과 마찬가지로 수어 통역이 제공될 것으로 보입니다. $지난달 와이 티 엔 의 `수어 소통 없는 국회 소통관` 보도와 관련해 앞으로는 국회 기자회견에도 코로나 일 구 정부 브리핑과 마찬가지로 수어 통역이 제공될 것으로 보입니다.
YTNPO480:더불어민주당 김태년 원내대표는 아파트 투기 세력 근절에 모든 정책 수단을 동원하겠다고 밝혔습니다. $더불어민주당 김태년 원내대표는 아파트 투기 세력 근절에 모든 정책 수단을 동원하겠다고 밝혔습니다.
YTNPO480:김 원내대표는 오늘 원내대책회의에서 지난해 12 , 16 부동산 대책에서 고강도 정책을 발표했지만, 종합부동산세 강화 등을 담은 후속 법안이 20 대 국회에서 처리되지 않아 시장에서 부작용이 발생하고 있다고 지적했습니다. $김 원내대표는 오늘 원내대책회의에서 지난해 십 이 , 십 육 부동산 대책에서 고강도 정책을 발표했지만, 종합부동산세 강화 등을 담은 후속 법안이 이십 대 국회에서 처리되지 않아 시장에서 부작용이 